In [ ]:
import numpy as np 
import pandas as pd 
%matplotlib inline
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import re
import seaborn as sns
from IPython.display import display
pd.options.mode.chained_assignment = None
import matplotlib
matplotlib.style.use('ggplot')
!pip install nltk
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')



from google.colab import drive
drive.mount('/content/drive')
tweets = pd.read_csv('/content/drive/My Drive/dataset2.csv', nrows=500,encoding = "ISO-8859-1")

!pip install vadersentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
display(tweets.head(3))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
import re

tweets['text_new'] = ''
tweets['tweetos'] = '' 


for i in range(len(tweets['text'][:500])):
    try:
        tweets['tweetos'][i] = tweets['text'].str.split(':')[i][0]
    except AttributeError:    
        tweets['tweetos'][i] = 'other'


for i in range(len(tweets['text'][:500])):
    if tweets['tweetos'].str.contains('RT @')[i]  == False:
        tweets['tweetos'][i] = 'other'


for i in range(len(tweets['text'][:500])):
    m = re.search('(?<=:)(.*)', tweets['text'][i])
    if tweets['text'].str.contains('RT @')[i]  == True:
        try:
            tweets['text_new'][i]=m.group(0)
        except AttributeError:
            tweets['text_new'][i]=tweets['text'][i] 
    else:       
        tweets['text_new'][i] =  tweets['text'][i]       

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

def wordcloud_by_province(tweets):
    stopwords = set(STOPWORDS)
    stopwords.add("https")
    stopwords.add("00A0")
    stopwords.add("00BD")
    stopwords.add("00B8")
    stopwords.add("ed")
    stopwords.add("happy")
    stopwords.add("Sad")
   
    stopwords.add("lakh")
    wordcloud = WordCloud(background_color="white",stopwords=stopwords,random_state = 2016).generate(" ".join([i for i in tweets['text_new'].str.upper()]))
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.title("Word cloud")

wordcloud_by_province(tweets)  

In [ ]:
print(tweets['text'].describe())

In [ ]:
tweets.describe()

In [ ]:
tweets['nb_words'] = 0
for i in range(len(tweets['text'])):
    tweets['nb_words'][i] = len(tweets['text'][i].split(' '))

In [ ]:
t=tweets['Sentiment'].value_counts()
t

In [ ]:
t.sort_values().plot(kind = 'barh')

In [ ]:
def get_stop_words(s, n):
	
	from collections import Counter
	l = get_corpus(s)
	l = [x for x in Counter(l).most_common(n)]
	return l

def get_corpus(s):
	
	l = []
	s.map(lambda x: l.extend(x))
	return l


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.stem import WordNetLemmatizer

tweets['text_lem'] = [''.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]', ' ', line)) for line in lists]).strip() for lists in tweets['text_new']]       

vectorizer = TfidfVectorizer(max_df=0.5,max_features=10000,min_df=10,stop_words='english',use_idf=True)
X = vectorizer.fit_transform(tweets['text_lem'].str.upper())
print(X.shape)


In [ ]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=5,init='k-means++',max_iter=200,n_init=1)

km.fit(X)
terms = vectorizer.get_feature_names()
order_centroids = km.cluster_centers_.argsort()[:,::-1]
for i in range(5):
    print("cluster %d:" %i, end='')
    for ind in order_centroids[i,:10]:
        print(' %s' % terms[ind], end='')
    print()    

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(X)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pos = pca.fit_transform(dist)
xs, ys = pos[:,0], pos[:,1]

In [ ]:

cluster_colors = {0: '#1b9e77', 1: '#d95f02', 2: '#7570b3', 3: '#e7298a', 4: '#66a61e'}#, 5: '#8A2BE2', 6: '#E9967A'}

cluster_names = {0: 'cluster 1', 
                 1: 'cluster 2', 
                 2: 'cluster 3', 
                 3: 'cluster 4', 
                 4: 'cluster 5'}
                 #5: 'cluster 6',
                 #6: 'cluster 7'}
clusters = km.labels_.tolist()

In [ ]:

%matplotlib inline 

df = pd.DataFrame(dict(x=xs, y=ys, label=clusters, title='')) 


groups = df.groupby('label')



fig, ax = plt.subplots(figsize=(10, 4)) # set size
ax.margins(0.05) 


for name, group in groups:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=10, 
            label=cluster_names[name], color=cluster_colors[name],
            mec='none')
    ax.set_aspect('auto')
    ax.tick_params(\
        axis= 'x',          
        which='both',     
        bottom='off',      
        top='off',         
        labelbottom='off')
    ax.tick_params(\
        axis= 'y',        
        which='both',     
        left='off',      
        top='off',        
        labelleft='off')
    
ax.legend(numpoints=1)  
plt.title('Cluster plotting with ACP', bbox={'facecolor':'0.8', 'pad':0})

for i in range(len(df)):
    ax.text(df.iloc[i]['x'], df.iloc[i]['y'], df.iloc[i]['title'], size=5)  

    
    
plt.show()

In [ ]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from tablib.utils import *
from nltk import tokenize

sid = SentimentIntensityAnalyzer()

tweets['sentiment_compound_polarity']=tweets.text_lem.apply(lambda x:sid.polarity_scores(x)['compound'])
tweets['sentiment_neutral']=tweets.text_lem.apply(lambda x:sid.polarity_scores(x)['neu'])
tweets['sentiment_negative']=tweets.text_lem.apply(lambda x:sid.polarity_scores(x)['neg'])
tweets['sentiment_pos']=tweets.text_lem.apply(lambda x:sid.polarity_scores(x)['pos'])
tweets['sentiment_type']=''
tweets.loc[tweets.sentiment_compound_polarity>0,'sentiment_type']='POSITIVE'
tweets.loc[tweets.sentiment_compound_polarity==0,'sentiment_type']='NEUTRAL'
tweets.loc[tweets.sentiment_compound_polarity<0,'sentiment_type']='NEGATIVE'
tweets.head()

In [ ]:
from  sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score


tweets_num_mod = tweets[tweets.select_dtypes(exclude=['object']).columns.values]


In [ ]:

X,y=extract_feature(tweets_num_mod)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
ac=[]

In [ ]:
from keras import layers
from keras.models import Sequential

model = Sequential()
model.add(layers.Dense(20, input_dim=4, activation='relu'))
model.add(layers.Dense(10,  activation='tanh'))
model.add(layers.Dense(1024, activation='relu'))

model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
              optimizer='adam')
model.summary()
classifier_nn = model.fit(X_train,y_train,
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=15)
ac.append(accuracy_score(classifier_nn,y_test,sample_weight=0.2)*100)

In [ ]:
from sklearn.svm import SVC
clf=SVC(gamma='auto')
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
ac.append(accuracy_score(y_pred,y_test)*100)

In [ ]:

def predict(s):
    model=SentimentIntensityAnalyzer()
    scores=model.polarity_scores(s)
    c=comp(scores)
    return c
def comp(scores):
    if(scores['neg']>scores['pos'] and scores['neg']>scores['neu'] ):
        return "Predicted as depressed or sad tweet"

    elif(scores['pos']>scores['neg'] and scores['pos']>scores['neu'] ):
        return "Predicted as happy tweet"
    else:
        return "Neutral tweet"

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib as plt

plt.style.use('seaborn')
x=['CNN','SVM']
 
ax=sns.barplot(x,ac)
ax.set_title('Accuracy comparison')
ax.set_ylabel('Accuracy')

print("the accuracy of {} is {} and {} is {}".format(x[0],ac[0],x[1],ac[1]))
ax.set_ylim(50,100)
import pandas as pd
data={'Agorithms':x,
     "accuracy":ac}
df=pd.DataFrame(data)
df.head()

In [ ]:
predict("i woke up early morning")